Lesson 3: Enable Logging

In [8]:
import boto3
import json
import os

bedrock = boto3.client('bedrock', region_name="us-west-2")

In [9]:
from helpers.CloudWatchHelper import CloudWatch_Helper
cloudwatch = CloudWatch_Helper()

In [10]:
log_group_name = '/my/amazon/bedrock/logs'

In [11]:
cloudwatch.create_log_group(log_group_name)

Log group '/my/amazon/bedrock/logs' already exists.


In [12]:
loggingConfig = {
    'cloudWatchConfig': {
        'logGroupName': log_group_name,
        'roleArn': os.environ['LOGGINGROLEARN'],
        'largeDataDeliveryS3Config': {
            'bucketName': os.environ['LOGGINGBUCKETNAME'],
            'keyPrefix': 'amazon_bedrock_large_data_delivery',
        }
    },
    's3Config': {
        'bucketName': os.environ['LOGGINGBUCKETNAME'],
        'keyPrefix': 'amazon_bedrock_logs',
    },
    'textDataDeliveryEnabled': True,
}

In [13]:
bedrock.put_model_invocation_logging_configuration(loggingConfig=loggingConfig)

{'ResponseMetadata': {'RequestId': 'e1b33d6e-56ad-4e6b-85a1-2d8b34331813',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 15 Feb 2024 13:52:24 GMT',
   'content-type': 'application/json',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'e1b33d6e-56ad-4e6b-85a1-2d8b34331813'},
  'RetryAttempts': 0}}

In [14]:
bedrock.get_model_invocation_logging_configuration()

{'ResponseMetadata': {'RequestId': 'cf63ae00-2d7f-4c4d-93fb-ad655f93c8fb',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 15 Feb 2024 13:52:50 GMT',
   'content-type': 'application/json',
   'content-length': '480',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'cf63ae00-2d7f-4c4d-93fb-ad655f93c8fb'},
  'RetryAttempts': 0},
 'loggingConfig': {'cloudWatchConfig': {'logGroupName': '/my/amazon/bedrock/logs',
   'roleArn': 'arn:aws:iam::484494363864:role/DLAITestingIAMRoleStack-MyIAMRole-QKDbeb7FXvMv',
   'largeDataDeliveryS3Config': {'bucketName': 'dlai-bucket-logging-owa002',
    'keyPrefix': 'amazon_bedrock_large_data_delivery'}},
  's3Config': {'bucketName': 'dlai-bucket-logging-owa002',
   'keyPrefix': 'amazon_bedrock_logs'},
  'textDataDeliveryEnabled': True,
  'imageDataDeliveryEnabled': True,
  'embeddingDataDeliveryEnabled': True}}

In [15]:
bedrock_runtime = boto3.client('bedrock-runtime', region_name="us-west-2")

In [16]:
prompt = "Write an article about the fictional planet Foobar."

kwargs = {
    "modelId": "amazon.titan-text-express-v1",
    "contentType": "application/json",
    "accept": "*/*",
    "body": json.dumps(
        {
            "inputText": prompt,
            "textGenerationConfig": {
                "maxTokenCount": 512,
                "temperature": 0.7,
                "topP": 0.9
            }
        }
    )
}

response = bedrock_runtime.invoke_model(**kwargs)
response_body = json.loads(response.get('body').read())

generation = response_body['results'][0]['outputText']

print(generation)


Foobar is a fictional planet in the Star Wars universe. It is the primary setting of the Star Wars: The Phantom Menace novel and film. The planet is a gas giant with a thick atmosphere and a surface covered in oceans and deserts. The planet is home to a variety of sentient species, including the Trade Federation, the Gungans, and the Naboo.

One of the most notable features of Foobar is its trade network. The planet is a major hub for trade in the Outer Rim, with a wide variety of goods and services available for purchase. The Trade Federation, a powerful conglomerate of corporations, controls most of the planet's trade, and its members are known for their strict adherence to trade regulations.

Another important feature of Foobar is the Gungan race. The Gungan are a species of aquatic humanoids with a unique culture and language. They are known for their bravery and their advanced technology, particularly in the field of underwater exploration. The Gungans are allies of the Naboo, an

In [17]:
cloudwatch.print_recent_logs(log_group_name)

Permissions are correctly set for Amazon Bedrock logs.
-------------------------

{
    "schemaType": "ModelInvocationLog",
    "schemaVersion": "1.0",
    "timestamp": "2024-02-15T13:53:10Z",
    "accountId": "484494363864",
    "identity": {
        "arn": "arn:aws:iam::484494363864:user/bedrock"
    },
    "region": "us-west-2",
    "requestId": "26076f0f-b967-4c53-8725-27f42c553a23",
    "operation": "InvokeModel",
    "modelId": "amazon.titan-text-express-v1",
    "input": {
        "inputContentType": "application/json",
        "inputBodyJson": {
            "inputText": "Write an article about the fictional planet Foobar.",
            "textGenerationConfig": {
                "maxTokenCount": 512,
                "temperature": 0.7,
                "topP": 0.9
            }
        },
        "inputTokenCount": 10
    },
    "output": {
        "outputContentType": "application/json",
        "outputBodyJson": {
            "inputTextTokenCount": 10,
            "results": [
 